In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('c:/fomo_data2/model_data_weather.parquet')

In [3]:
df.head()

,locationid,ridership,year,month,day,hour,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2.0,1.0,2022,2,6,15,-5.6,0.0,0.0,11.0,0.0,0.0,35.0,6.600000,331.0
1,2.0,1.0,2022,2,16,23,5.4,0.0,0.0,29.0,0.0,0.0,97.0,12.100000,280.0
2,2.0,1.0,2022,2,23,8,13.5,0.0,0.0,44.0,8.0,11.0,100.0,16.299999,227.0
3,2.0,4.0,2022,2,23,19,17.5,0.0,0.0,22.0,5.0,2.0,54.0,21.600000,184.0
4,2.0,1.0,2022,3,22,10,7.4,0.0,0.0,20.0,0.0,30.0,8.0,10.800000,127.0


In [4]:
import pandas as pd
comments = pd.read_csv('c:/fomo_data/sentiment_scores1.csv')

In [5]:
comments.head()

,id,name,address,comment,created_at,latitude,longitude,sentiment
0,1,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281",hot yoga point . clear view across hudson weig...,2019-12-31 17:12:40,40.711974,-74.016237,1
1,2,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281","gym floor bit uninspiring , location great .",2019-12-17 14:00:01,40.711974,-74.016237,1
2,3,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281",location low ceiling get crowded morning,2019-01-14 04:06:52,40.711974,-74.016237,-1
3,4,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281",excellent ! class motivate well great equipment,2018-09-25 10:38:14,40.711974,-74.016237,1
4,5,Equinox Brookfield Place,"225 Liberty St (at West St), New York, NY 10281","great view nice lounge area , overall pretty s...",2018-07-10 22:41:19,40.711974,-74.016237,1


In [6]:
comments = comments.drop(columns=['name', 'address', 'comment'])
comments.head()

,id,created_at,latitude,longitude,sentiment
0,1,2019-12-31 17:12:40,40.711974,-74.016237,1
1,2,2019-12-17 14:00:01,40.711974,-74.016237,1
2,3,2019-01-14 04:06:52,40.711974,-74.016237,-1
3,4,2018-09-25 10:38:14,40.711974,-74.016237,1
4,5,2018-07-10 22:41:19,40.711974,-74.016237,1


In [7]:
comments = comments.drop(columns=['id'])

In [8]:
import geopandas as gpd

#sahpe file for the taxi zones
taxi_zones = gpd.read_file('c:/fomo_data/taxi.shp')

# Convert subway latitude and longitude to a GeoDataFrame
geometry = gpd.points_from_xy(comments['longitude'], comments['latitude'])
comments_geo = gpd.GeoDataFrame(comments, geometry=geometry, crs="EPSG:4326")

# Reproject taxi_zones to the same CRS as subway_geo if needed
taxi_zones = taxi_zones.to_crs(comments_geo.crs)

# Perform a spatial join to find the taxi zone for each subway station using 'predicate' instead of 'op'
subway_geo = gpd.sjoin(comments_geo, taxi_zones, how='left', predicate='intersects')

In [9]:
subway_geo.head()

,created_at,latitude,longitude,sentiment,geometry,index_right,borough,location_i,objectid,shape_area,shape_leng,zone
0,2019-12-31 17:12:40,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
1,2019-12-17 14:00:01,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
2,2019-01-14 04:06:52,40.711974,-74.016237,-1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
3,2018-09-25 10:38:14,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City
4,2018-07-10 22:41:19,40.711974,-74.016237,1,POINT (-74.01624 40.71197),13.0,Manhattan,13.0,13.0,0.000149,0.050281,Battery Park City


In [26]:
comments['location_id'] = subway_geo['location_i']

In [11]:
comments.head()

,created_at,latitude,longitude,sentiment,location_id
0,2019-12-31 17:12:40,40.711974,-74.016237,1,13.0
1,2019-12-17 14:00:01,40.711974,-74.016237,1,13.0
2,2019-01-14 04:06:52,40.711974,-74.016237,-1,13.0
3,2018-09-25 10:38:14,40.711974,-74.016237,1,13.0
4,2018-07-10 22:41:19,40.711974,-74.016237,1,13.0


In [5]:
comments = pd.read_parquet('c:/fomo_data/comments_cleaned.parquet')

In [10]:
comments.drop(columns=['latitude', 'longitude'])

,created_at,sentiment
0,2019-12-31 17:12:40,1
1,2019-12-17 14:00:01,1
2,2019-01-14 04:06:52,-1
3,2018-09-25 10:38:14,1
4,2018-07-10 22:41:19,1
...,...,...
18156,2018-04-06 15:50:32,1
18157,2024-02-15 00:58:10,0
18158,2018-09-21 19:46:45,0
18159,2017-09-07 00:06:36,1


In [11]:
df.head()

,locationid,ridership,year,month,day,hour,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2.0,1.0,2022,2,6,15,-5.6,0.0,0.0,11.0,0.0,0.0,35.0,6.600000,331.0
1,2.0,1.0,2022,2,16,23,5.4,0.0,0.0,29.0,0.0,0.0,97.0,12.100000,280.0
2,2.0,1.0,2022,2,23,8,13.5,0.0,0.0,44.0,8.0,11.0,100.0,16.299999,227.0
3,2.0,4.0,2022,2,23,19,17.5,0.0,0.0,22.0,5.0,2.0,54.0,21.600000,184.0
4,2.0,1.0,2022,3,22,10,7.4,0.0,0.0,20.0,0.0,30.0,8.0,10.800000,127.0


In [12]:
df = df.drop(columns=['rain (mm)', 'precipitation (mm)', 'cloudcover_low (%)', 'cloudcover (%)', 'cloudcover_mid (%)', 'cloudcover_high (%)' ])

In [13]:
df.head()

,locationid,ridership,year,month,day,hour,temperature_2m (°C),windspeed_10m (km/h),winddirection_10m (°)
0,2.0,1.0,2022,2,6,15,-5.6,6.600000,331.0
1,2.0,1.0,2022,2,16,23,5.4,12.100000,280.0
2,2.0,1.0,2022,2,23,8,13.5,16.299999,227.0
3,2.0,4.0,2022,2,23,19,17.5,21.600000,184.0
4,2.0,1.0,2022,3,22,10,7.4,10.800000,127.0


In [14]:
print(len(df))

3856928


In [15]:
comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18161 entries, 0 to 18160
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   created_at  18161 non-null  object 
 1   latitude    18161 non-null  float64
 2   longitude   18161 non-null  float64
 3   sentiment   18161 non-null  int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 567.7+ KB


In [16]:
comments['created_at'] = comments['created_at'].astype('category')
comments['created_at'] = pd.to_datetime(comments['created_at'])
comments['month'] = comments['created_at'].dt.month
comments['day_of_week'] = comments['created_at'].dt.day
comments.drop(columns=['created_at'])
comments.head()

,created_at,latitude,longitude,sentiment,month,day_of_week
0,2019-12-31 17:12:40,40.711974,-74.016237,1,12,31
1,2019-12-17 14:00:01,40.711974,-74.016237,1,12,17
2,2019-01-14 04:06:52,40.711974,-74.016237,-1,1,14
3,2018-09-25 10:38:14,40.711974,-74.016237,1,9,25
4,2018-07-10 22:41:19,40.711974,-74.016237,1,7,10


In [17]:
comments.drop(columns=['created_at'], inplace=True)

In [18]:
comments.head()

,latitude,longitude,sentiment,month,day_of_week
0,40.711974,-74.016237,1,12,31
1,40.711974,-74.016237,1,12,17
2,40.711974,-74.016237,-1,1,14
3,40.711974,-74.016237,1,9,25
4,40.711974,-74.016237,1,7,10


In [19]:
comments.drop(columns=['longitude', 'latitude'], inplace=True)
comments.head()

,sentiment,month,day_of_week
0,1,12,31
1,1,12,17
2,-1,1,14
3,1,9,25
4,1,7,10


In [20]:
comments.rename(columns={'location_id': 'locationid'}, inplace=True)
comments.rename(columns={'day_of_week': 'day'}, inplace=True)

In [21]:
df.nunique()

locationid                 260
ridership                29629
year                         3
month                       12
day                         31
hour                        24
temperature_2m (°C)        444
windspeed_10m (km/h)       340
winddirection_10m (°)      360
dtype: int64

In [22]:
print(len(df))

3856928


In [27]:
comments.head()

,sentiment,month,day,location_id
0,1,12,31,13.0
1,1,12,17,13.0
2,-1,1,14,13.0
3,1,9,25,13.0
4,1,7,10,13.0


In [30]:
# Check for duplicate keys in df
duplicate_keys_df = df[df.duplicated(subset=['locationid', 'month', 'day'], keep=False)]
print("Duplicate keys in df:")
print(duplicate_keys_df)

# Check for duplicate keys in comments
duplicate_keys_comments = comments[comments.duplicated(subset=['location_id', 'month', 'day'], keep=False)]
print("Duplicate keys in comments:")
print(duplicate_keys_comments)

Duplicate keys in df:
         locationid  ridership  year  month  day  hour  temperature_2m (°C)  \
0               2.0        1.0  2022      2    6    15                 -5.6   
1               2.0        1.0  2022      2   16    23                  5.4   
2               2.0        1.0  2022      2   23     8                 13.5   
3               2.0        4.0  2022      2   23    19                 17.5   
4               2.0        1.0  2022      3   22    10                  7.4   
...             ...        ...   ...    ...  ...   ...                  ...   
3856923       263.0      985.0  2024      5   30    19                  NaN   
3856924       263.0      573.0  2024      5   30    20                  NaN   
3856925       263.0      452.0  2024      5   30    21                  NaN   
3856926       263.0      370.0  2024      5   30    22                  NaN   
3856927       263.0      226.0  2024      5   30    23                  NaN   

         windspeed_10m (km/h)

In [36]:
comments.rename(columns={'location_id': 'locationid'}, inplace=True)
comments.head()

,sentiment,month,day,locationid
0,1,12,31,13.0
1,1,12,17,13.0
2,-1,1,14,13.0
3,1,9,25,13.0
4,1,7,10,13.0


In [40]:
comments_unique = comments.groupby(['locationid', 'month', 'day']).agg({'sentiment': 'mean'}).reset_index()

# Merge df and comments_unique on locationid, month, and day
merged_df = pd.merge(df, comments_unique, on=['locationid', 'month', 'day'], how='left')

# Fill NaN values in the 'sentiment' column with 0
merged_df['sentiment'] = merged_df['sentiment'].fillna(0)

# Verify the merged DataFrame
print("Merged DataFrame shape:", merged_df.shape)
print(merged_df)

Merged DataFrame shape: (3856928, 10)
         locationid  ridership  year  month  day  hour  temperature_2m (°C)  \
0               2.0        1.0  2022      2    6    15                 -5.6   
1               2.0        1.0  2022      2   16    23                  5.4   
2               2.0        1.0  2022      2   23     8                 13.5   
3               2.0        4.0  2022      2   23    19                 17.5   
4               2.0        1.0  2022      3   22    10                  7.4   
...             ...        ...   ...    ...  ...   ...                  ...   
3856923       263.0      985.0  2024      5   30    19                  NaN   
3856924       263.0      573.0  2024      5   30    20                  NaN   
3856925       263.0      452.0  2024      5   30    21                  NaN   
3856926       263.0      370.0  2024      5   30    22                  NaN   
3856927       263.0      226.0  2024      5   30    23                  NaN   

         wind

In [41]:
sentiment_description = merged_df['sentiment'].describe()
print("Descriptive Statistics of 'sentiment' column:")
print(sentiment_description)

# Check for missing values in the 'sentiment' column
missing_sentiment = merged_df['sentiment'].isna().sum()
print("\nNumber of missing values in 'sentiment' column:", missing_sentiment)

# Distribution of the 'sentiment' column
sentiment_distribution = merged_df['sentiment'].value_counts()
print("\nDistribution of 'sentiment' column values:")
print(sentiment_distribution)

# Additional info about the 'sentiment' column
print("\nAdditional info about the 'sentiment' column:")
print(merged_df['sentiment'].info())

Descriptive Statistics of 'sentiment' column:
count    3.856928e+06
mean     4.274998e-02
std      2.173516e-01
min     -1.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.000000e+00
Name: sentiment, dtype: float64

Number of missing values in 'sentiment' column: 0

Distribution of 'sentiment' column values:
sentiment
 0.000000    3631113
 1.000000     157747
 0.500000      19090
-1.000000      16251
 0.666667       7214
              ...   
-0.285714         48
 0.555556         48
 0.567568         48
 0.583333         47
-0.142857         11
Name: count, Length: 69, dtype: int64

Additional info about the 'sentiment' column:
<class 'pandas.core.series.Series'>
RangeIndex: 3856928 entries, 0 to 3856927
Series name: sentiment
Non-Null Count    Dtype  
--------------    -----  
3856928 non-null  float64
dtypes: float64(1)
memory usage: 29.4 MB
None


In [42]:
merged_df.to_parquet('c:/fomo_data2/comments_merged_fixed.parquet')

In [43]:
print(len(merged_df))

3856928


In [44]:
nan_by_columns = merged_df.isna().sum()
print("NaN values by columns:")
print(nan_by_columns)

# Check for NaN values by rows
nan_by_rows = merged_df.isna().sum(axis=1)
print("\nNaN values by rows:")
print(nan_by_rows)

NaN values by columns:
locationid                    0
ridership                     0
year                          0
month                         0
day                           0
hour                          0
temperature_2m (°C)      634246
windspeed_10m (km/h)     634246
winddirection_10m (°)    634246
sentiment                     0
dtype: int64

NaN values by rows:
0          0
1          0
2          0
3          0
4          0
          ..
3856923    3
3856924    3
3856925    3
3856926    3
3856927    3
Length: 3856928, dtype: int64


In [37]:
merged_df = merged_df.dropna(how='all')

In [46]:
print(len(merged_df))

3222682


In [45]:
merged_df= merged_df.dropna()

In [48]:
merged_df.to_parquet('c:/fomo_data2/fixed_with_comments.parquet')